<a href="https://colab.research.google.com/github/tanzid2059/Bangla-Sentiment-Analysis-using-semi-supervise-learning-model/blob/main/BanglaSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random
import csv
from sklearn.decomposition import PCA

In [ ]:
!pip install -q wordcloud
import wordcloud

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
pip install bangla-stemmer

In [ ]:
import matplotlib.pyplot as plt
import io
import unicodedata
import re
import string

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving 1.txt to 1.txt


In [ ]:
unlabel_data=pd.read_csv("/content/unlabel data.txt",sep="\t",names=['Bangla_Sentences'])

In [ ]:
unlabel_data.head(10)

,Bangla_Sentences
0,সে ঘুরে ল্যাংডনের চোখের দিকে তাকায়।
1,সব মানুষের মধ্যে সমতা আনতে হলে কারও না কারও স্...
2,"যাই জিজ্ঞেস কর না কেন, সব প্রশ্নের জবাব দিব ডি..."
3,ঠিক যখন আমি সভাকে শুভরাত জানিয়ে বিদায় নিতে উদ্...
4,যেতে দিন!
5,"এবার ক্যামারলেনগো ঘুরে দাঁড়াল আবার, তাকাল অন্য..."
6,তাদের ট্যাক্সি এখন কোথায়?
7,"বেশিরভাগ গবেষকই মনে করেন যে, এতসব গুরুত্বপূর্ণ..."
8,দশকের পর দশক ধরে এসব প্রাণীর জীবাশ্ম আর দেহাবশ...
9,এটা প্রাপ্ত বয়স্কদের জন্য এক রকমের স্কুলও বলা ...


In [ ]:
#from sklearn.model_selection import train_test_split

#train_data,test_data=train_test_split(data,test_size=0.2, random_state=1)

# **Pre-Processing Functions**

In [ ]:
def cleanDataset(src_path, des_path):
  sentences = []
  # Using readline() 
  count = 0
  with open(src_path,'r',encoding = 'utf-8') as f:
    for line in f:
      if len(line) >1 :      
        count += 1
        sentences.append(line)
  
  l = process(sentences)
  # print(l[0:10])
  f = open(des_path, "a")
  for s in l:
    f.write(s+"\n")
  f.close()

In [ ]:
def process(sentences):
    
  l=[]
  numOfConsecutiveSentence = 2
  minWordLimit = 3 #including ?,|,!
  maxWordLimit = 15 #including ?,|,!
  kk = 0
  for text in sentences:
    kk+=1
    c = 0
    s = ""
    mergedSentecnes = ""
    oneSetenceFound = False
    numOfSentenceMerged = 0
    for i  in  range(0,len(text)):
      
      # removes starting spaces
      if len(s) == 0 and text[i] == ' ':
        continue

      # replace charcter with ' ' if character is not bangla character or number
      if text[i] == '-' or text[i] == ':' or text[i] == '/' : 
        if len(s) > 1 and i < len(text)-1:
          if isOkkhor(s[len(s)-1]) and isOkkhor(text[i+1]):
            s += ' '
      # if charcter is bangla character or number or ' '
      elif isCorrect(text[i]):
        if len(s) > 1 and text[i] == ' ': # remove consecutive space
          if s[len(s)-1] == ' ': 
            continue
        # end of a sentece 
        if text[i] == '।' or text[i] == '?' or text[i] == '!':
          if len(s) == 0:
            continue
          c+=1
          oneSetenceFound = True
          # add space before ?,!,। 
          
          if s[len(s)-1] != ' ':
            s += ' ' 

        s += text[i]

      if oneSetenceFound:
        oneSetenceFound = False
        wordCount= countW(s)
        if wordCount > minWordLimit-1 and wordCount < maxWordLimit:
          # add space after previous sentence
          if numOfSentenceMerged > 0:
            mergedSentecnes += (' ' + s)
          else:
            mergedSentecnes += s
          s = ""
          numOfSentenceMerged += 1
          # if sentece violates word limit ignore all merged sentence
        else:
          mergedSentecnes = ""
          s = ""
          numOfSentenceMerged = 0
      
      if numOfSentenceMerged == numOfConsecutiveSentence:

        c = 0
        numOfSentenceMerged = 0 
        mergedSentecnes = replaceNumber(mergedSentecnes)
        l.append(mergedSentecnes)
        mergedSentecnes = ""
        s = ""
  return l

In [ ]:
def replaceNumber(s):
   bangla_number = ['১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯', '০']
   newSentence = ""
   flag = False
   for i in range(0,len(s)):
     if s[i] in bangla_number and not flag:
       flag = True
      #  print(newSentence)
       if len(newSentence) == 0:
        newSentence += "11111111"
       elif newSentence[len(newSentence)-1] != ' ':
         newSentence += " 11111111"
       else:
         newSentence += "11111111"
     elif s[i] not in bangla_number:
       if i>0 and newSentence[len(newSentence)-1] == '1' and s[i] != ' ':
         newSentence += ' '
       flag = False
       newSentence += s[i]
   return newSentence

In [ ]:
def countW(s):
  c = 0
  c1 = 0
  first_sen = False
  for i in range(0,len(s)):
    if s[i] == ' ':
      c+=1
    
    if s[i] == '।' or s[i] == '?' or s[i] == '!':
      
      if s[i-1] != ' ':
        c+=1 

  return (c)

In [ ]:
def isCorrect(h):
  dic_char = ['া','র','ে','্','ি','ন','ক', 'ব', 'ত','স', 'ল', 'ম', 'প', 'ু', 'দ', 'য়', 'ট', 'জ', 'য', 'ো', 'শ','গ','হ','ই','ী', 'চ', 'ভ','ছ', 'আ', 'ষ', 'অ', 'ধ', 'খ',
              'ড', 'ও', 'ফ', 'ড়','উ','ণ','থ', 'ং','এ', 'ঁ', 'ূ', 'ঠ','ৃ','ঘ','ঞ','ঙ','়','ঝ','ৌ','ৎ','ৈ','ঢ','ঃ','ৗ','ঈ','ঐ','ঋ','ঢ়','ঊ','ঔ',
              '।','?','!',' ', '২','০','১','৯','৫','৭','৩','৪','৮','৬',' ']
  if h not in dic_char:
    return False
  else:
    return True

In [ ]:
def isOkkhor(h):
  dic_char = ['া','র','ে','্','ি','ন','ক', 'ব', 'ত','স', 'ল', 'ম', 'প', 'ু', 'দ', 'য়', 'ট', 'জ', 'য', 'ো','শ','গ','হ','ই','ী', 'চ', 'ভ','ছ', 'আ','ষ','অ','ধ','খ',
              'ড','ও','ফ','ড়','উ','ণ','থ','ং','এ','ঁ','ূ','ঠ','ৃ','ঘ','ঞ','ঙ','়','ঝ','ৌ','ৎ','ৈ','ঢ','ঃ','ৗ','ঈ','ঐ','ঋ','ঢ়','ঊ','ঔ',]
  if h not in dic_char:
    return False
  else:
    return True

In [ ]:
src_path='/content/all_negative.txt'
des_path='/content/processed_all_negative.txt'
cleanDataset(src_path=src_path, des_path=des_path)

src_path='/content/all_positive.txt'
des_path='/content/processed_all_positive.txt'
cleanDataset(src_path=src_path, des_path=des_path)

src_path='/content/unlabel data.txt'
des_path='/content/processed_unlabel_data.txt'
cleanDataset(src_path=src_path, des_path=des_path)

In [ ]:
processed_neg=pd.read_csv("/content/processed_all_negative.txt",sep="\t",names=['Processed_Bangla_Sentences'])
processed_pos=pd.read_csv("/content/processed_all_positive.txt",sep="\t",names=['Processed_Bangla_Sentences'])

In [ ]:
label_pos_data.shape

pos_sentiment=[1]*2000

label_pos_data['sentiment']=pos_sentiment
label_pos_data.head(10)

,Bangla_Sentences,sentiment
0,আমার খুব প্রিয় মডেল আমি খুব ভালো বাসি মিম আপু,1
1,ভাই সব আপনাদের খুব ভাল লাগছে,1
2,আপু তুমি অনেক ন্যচারাল সুন্দর,1
3,রাজশাহী শিরইলে অনেক রকমের পাখি আসে আমি দেখছি ...,1
4,তাদের কত সম্মান করি আমরা,1
5,ইন্নালিল্লাহে ওয়া ইন্নাইলাহি রাজিউন। আল্লাহ উ...,1
6,এদেশে যতই জনসংখ্যা বেশি হোক তারপরেও জীবন বাঁচ...,1
7,নতুন সিমে প্রথমবার টাকা রিচার্জে আপনি মিনিট ব...,1
8,বি বি সি কে অনেক ধন্যবাদ।,1
9,বাবুটা তার সাথে আবু হেবি,1


In [ ]:
train_data_array = []
for i in range(len(label_data)):
    train_data_array.append(label_data["Bangla_Sentences"][i])

train_data_list = [] 
for d in train_data_array:
    x = d.split()
    sentence_list = []
    for y in x:
        split_word = (y).split('|')
        word = split_word[0]
        sentence_list.append(word)
    train_data_list.append(sentence_list)

In [ ]:
label_neg_data.shape
neg_sentiment=[0]*2000

label_neg_data['sentiment']=neg_sentiment
label_neg_data.head(10)

,Bangla_Sentences,sentiment
0,জগন্নাথের শিক্ষার্থীরা এটা মেনে নিবে না। এটা ...,0
1,মাননীয় মহাপরিদর্শক পুলিশ আমাদেরকে ট্রাফিক পুল...,0
2,চুপ থাকার কোন সিস্টেম নাই যদি প্রান খুলে হাসত...,0
3,আবারও লেট হয়ছে সেকেন্ড মাত্র বাহ বাহ,0
4,ভাই সাংবাদিক পুলিশ যখন খারাপ কিছু করে তখন পুল...,0
5,আমার মতে এই তিন জনকে জেলে রেখে বোজানো হক পাখি...,0
6,এই শালাগুলোকে কাঁচা কঞ্চি দিয়ে পিটানো দরকার ত...,0
7,কিছু কিছু পাক প্রেমি কথিত বাংলাদেশীআদতে বিহার...,0
8,হ্যাঁ তাই ছোটভাইকে হত্যা করা উচিত সবদিক থেকে ...,0
9,পাকিস্তানের পাশে আপনি অবশ্যই থাকবেন। পিতার দেশ...,0


In [ ]:
label_data = pd.concat([label_neg_data,label_pos_data], ignore_index=True)

In [ ]:
label_data.tail(10)

,Bangla_Sentences,sentiment
3990,তোমার বাবা সাথে তোলা ছবি খুব সুন্দর,1
3991,চেয়েছিলাম পাঁচ উইকেট সেটা পেয়েছো এবার ব্যাট হ...,1
3992,এগিয়ে যাও সাকিব ভাই এগিয়ে যাক বাংলাদেশ।,1
3993,স্থান পরিবতন করা গেলে ভাল হয়।,1
3994,আমরা রাষ্টহিসাবে ছোট হলেও আমারা মুসলিম জাতি ব...,1
3995,নয়া দিগন্তকে অনুরোধ আমাদের কথাগুলো আপনারা গুর...,1
3996,জাহিদ ইমন আপনার মতামতটি আমাদের সাথে শেয়ার করা...,1
3997,একটা মেয়ের জন্যে তোমাকে পারফেক্ট হওয়ার কোন ...,1
3998,এ ঘটনা বার বার ঘটছে । কি ভাবে থামনো যায় তার চ...,1
3999,অদের খাবার আর সব কিছু দিয়া সাহায্য করা,1


In [ ]:
label_data.to_csv('/content/processed_label_data.txt')

In [ ]:
x_label_data=label_data['Bangla_Sentences']
y_label_data=label_data['sentiment']

In [ ]:
print(x_label_data)

0        জগন্নাথের শিক্ষার্থীরা এটা মেনে নিবে না। এটা ...
1        মাননীয় মহাপরিদর্শক পুলিশ আমাদেরকে ট্রাফিক পুল...
2        চুপ থাকার কোন সিস্টেম নাই যদি প্রান খুলে হাসত...
3                   আবারও লেট হয়ছে সেকেন্ড মাত্র বাহ বাহ 
4        ভাই সাংবাদিক পুলিশ যখন খারাপ কিছু করে তখন পুল...
                              ...                        
3995     নয়া দিগন্তকে অনুরোধ আমাদের কথাগুলো আপনারা গুর...
3996     জাহিদ ইমন আপনার মতামতটি আমাদের সাথে শেয়ার করা...
3997     একটা মেয়ের জন্যে তোমাকে পারফেক্ট হওয়ার কোন ...
3998     এ ঘটনা বার বার ঘটছে । কি ভাবে থামনো যায় তার চ...
3999              অদের খাবার আর সব কিছু দিয়া সাহায্য করা 
Name: Bangla_Sentences, Length: 4000, dtype: object


In [ ]:
from sklearn.model_selection import train_test_split

x_label_train_data,x_label_test_data,y_label_train_data,y_label_test_data=train_test_split(x_label_data,y_label_data,test_size=0.2, stratify=y_label_data)

In [ ]:
from sklearn.model_selection import train_test_split

x_unlabel_train_data,x_unlabel_test_data=train_test_split(unlabel_data,test_size=0.2)

In [ ]:
x_label_train_data[2]

' চুপ থাকার কোন সিস্টেম নাই যদি প্রান খুলে হাসতে চান তাহলে লাইক দেন আমাদের '

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer()
x_label_train_vectors = vectorizer.fit_transform(x_label_train_data)

vectorizer = CountVectorizer()
x_label_test_vectors = vectorizer.fit_transform(x_label_test_data)



In [ ]:
vectorizer = CountVectorizer()
x_unlabel_test_vectors = vectorizer.fit_transform(x_unlabel_test_data)

vectorizer = CountVectorizer()
x_unlabel_train_vectors = vectorizer.fit_transform(x_unlabel_train_data)

In [ ]:
print(x_label_train_vectors.shape)
print(x_label_test_vectors.shape)

(3200, 1909)
(800, 994)


In [ ]:
print(x_label_test_vectors.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
print(x_label_test_vectors.toarray()[13])

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model1 = DecisionTreeClassifier()
model1.fit(x_label_train_vectors,y_label_train_data)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
predDT = model1.predict(x_unlabel_train_vectors)